In [24]:
#  데이터 처리 모듈
import pandas as pd
from sklearn.impute import KNNImputer
# 행렬 등 통계 연산 모듈
import numpy as np
# 지수형 표기법 e를 연속형 변환
pd.options.display.float_format = '{:.4f}'.format
# 타입 어노테이션(Any, Sequence 등의 메서드 활용)
from typing import *
import os

# 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")
import os

# 조합 찾는 반복문 메서드
from itertools import permutations
from itertools import combinations

# 데이터 시각화
import matplotlib.pyplot as plt
# from matplotlib.pyplot import
import matplotlib.font_manager as fm
import missingno as msno
import seaborn as sns

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
# apt-get : 패키지 설치 명령어
# -qq : 에러외의 메세지 숨기는 옵션키
!apt-get -qq install fonts-nanum

fe = fm.FontEntry(
    fname=r"/usr/share/fonts/truetype/nanum/NanumGothic.ttf",
    # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')
# 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)
# Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'})
# 폰트 설정

In [26]:
class LoadGoogleDriveData():
  def __init__(self, data = None):
    self.data = data

  def loadData(self, file_path: str, file_name_extension,
               columnTF: bool, unicode: str) -> pd.DataFrame():
    self.data = pd.read_csv(os.path.join(file_path + file_name_extension),
                            index_col = columnTF,
                            na_values = "NaN",
                            encoding = unicode)
    return self.data

  # 용량이 큰 csv 파일 읽어오기(fopen - fread와 유사한 방식)
  def loadDataWithChunking(self, file_path: str, file_name_extension,
                           chunking_row_num: int, columnTF: bool, unicode: str) -> pd.DataFrame():
    chunkdata = pd.read_csv(os.path.join(file_path + file_name_extension),
                            chunksize = chunking_row_num,
                            index_col = columnTF,
                            na_values = "NaN",
                            encoding = unicode)
    self.data = list(chunkdata)
    return self.data


In [27]:
mountInstance = LoadGoogleDriveData()

In [28]:
CONTEST_2023 = mountInstance.loadDataWithChunking(
    file_path = "/content/drive/MyDrive/2023BigContest/data/contest data/", file_name_extension = "소상공인 매출등급 예상 대상 필지.csv",
    unicode = "cp949", chunking_row_num = 10**5, columnTF = False)

In [29]:
CONTEST_2022 = mountInstance.loadDataWithChunking(
    file_path = "/content/drive/MyDrive/2023BigContest/data/contest data/", file_name_extension = "필지단위 소상공인 매출등급 정보.csv",
    unicode = "cp949", chunking_row_num = 10**5, columnTF = False)

In [30]:
KCD_2022_2023 = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "SOHO_MERGE_DATA_LIST.csv",
    unicode = "utf-8", columnTF = False)

In [71]:
# 행정동코드 - 상권코드 매핑 데이터
MARKET_LOC_MAP = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/",
    file_name_extension = "market_loc_map.csv",
    unicode = "euc-kr", columnTF = False)

In [72]:
MARKET_LOC_MAP

,상권_코드_2022,상권_코드_2023,행정동_코드,법정동_코드,STDG_EMD_CD
0,2110008,3110008,11110515,11110101,11110101
1,2110008,3110008,11110515,11110102,11110102
2,2110008,3110008,11110515,11110103,11110103
3,2110008,3110008,11110515,11110104,11110104
4,2110008,3110008,11110515,11110105,11110105
...,...,...,...,...,...
3453,2130102,3130102,11230750,11230110,11230110
3454,2130166,3130166,11380590,11380107,11380107
3455,2130113,3130113,11260690,11260106,11260106
3456,2130124,3130124,11290780,11290138,11290138


In [95]:
# 텍스트 파일 n개가 포함된 C(로컬디스크) 내 파일경로 설정
file_path = "/content/drive/MyDrive/2023BigContest/data" + "/" + "상권_2022"
# 파일경로 내 텍스트 파일명을 디렉토리 리스트에 저장
directory_상권_2022 = os.listdir(file_path)
print(directory_상권_2022, end = ", ")

data_상권특성_2022: List = [
    (directory_상권_2022[index].split("(")[1]).split(")")[0]
    for index in range(len(directory_상권_2022))
]
print("\n", data_상권특성_2022, end = ", ")

['서울시 상권분석서비스(상권영역).csv', '서울시 상권분석서비스(상권배후지-아파트).csv', '서울시 상권분석서비스(상권-소득소비).csv', '서울시 상권분석서비스(상권-아파트).csv', '서울시 상권분석서비스(상권배후지-직장인구).csv', '서울시 상권분석서비스(상권배후지-점포).csv', '서울시 상권분석서비스(상권-점포).csv', '서울시 상권분석서비스(상권배후지-집객시설).csv', '서울시 상권분석서비스(버스 총 승차 승객수 정보).csv', '서울시 상권분석서비스(상권배후지-생활인구).csv', '서울시 상권분석서비스(지하철 총 승차 승객수 정보).csv', '서울시 상권분석서비스(상권-생활인구).csv', '서울시 상권분석서비스(상권-직장인구).csv', '서울시 상권분석서비스(행정동별 상권변화지표).csv', '서울시 상권분석서비스(상권배후지-소득소비).csv'], 
 ['상권영역', '상권배후지-아파트', '상권-소득소비', '상권-아파트', '상권배후지-직장인구', '상권배후지-점포', '상권-점포', '상

In [96]:
# 텍스트 파일 n개가 포함된 C(로컬디스크) 내 파일경로 설정
file_path = "/content/drive/MyDrive/2023BigContest/data" + "/" + "상권_2023"
# 파일경로 내 텍스트 파일명을 디렉토리 리스트에 저장
directory_상권_2023 = os.listdir(file_path)
print(directory_상권_2023, end = ",")

data_상권특성_2023: List = [
    (directory_상권_2023[index].split("(")[1]).split(")")[0]
    for index in range(len(directory_상권_2023))
]
print("\n", data_상권특성_2023, end = ", ")

['서울시 상권분석서비스(상권배후지-점포)_2023.csv', '서울시 상권분석서비스(상권-점포)_2023.csv', '서울시 상권분석서비스(행정동별 상권변화지표)_2023.csv', '서울시 상권분석서비스(상권배후지-직장인구)_2023.csv', '서울시 상권분석서비스(상권-직장인구)_2023.csv', '서울시 상권분석서비스(상권-아파트)_2023.csv', '서울시 상권분석서비스(상권배후지-아파트)_2023.csv', '서울시 상권분석서비스(버스 총 승차 승객수 정보)_2023.csv', '서울시 상권분석서비스(지하철 총 승차 승객수 정보)_2023.csv', '서울시 상권분석서비스(상권배후지-생활인구)_2023.csv', '서울시 상권분석서비스(상권-생활인구)_2023.csv', '서울시 상권분석서비스(상권-소득소비)_2023.csv', '서울시 상권분석서비스(상권배후지-집객시설)_2023.csv'],
 ['상권배후지-점포', '상권-점포', '행정동별 상권변화지표', '상권배후지-직장인구', '상권-직장인구', '상권-아파트', '상권배후지-아파트', '버스 총 승차 승객수 정

In [98]:
# 상권특성_2022 및 상권특성_2023 데이터의 교집합
print(np.intersect1d(data_상권특성_2022, data_상권특성_2023))
# 상권특성_2022 및 상권특성_2023 데이터의 여집합

['버스 총 승차 승객수 정보' '상권-생활인구' '상권-소득소비'
 '상권-아파트' '상권-점포' '상권-직장인구'
 '상권배후지-생활인구' '상권배후지-아파트' '상권배후지-점포'
 '상권배후지-직장인구' '상권배후지-집객시설'
 '지하철 총 승차 승객수 정보' '행정동별 상권변화지표']


In [100]:
print(np.setxor1d(data_상권특성_2022, data_상권특성_2023))

['상권배후지-소득소비' '상권영역']


In [101]:
directory_상권_2022.remove("서울시 상권분석서비스(상권배후지-소득소비).csv")
data_상권특성_2022.remove("상권배후지-소득소비")

In [103]:
MARKET_2022_LIST = []
for index, data in zip(list(range(len(directory_상권_2022))), directory_상권_2022):
  MARKET_2022 = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/상권_2022/",
    file_name_extension = data,
    unicode = "euc-kr", columnTF = False)
  MARKET_2022_LIST.append(MARKET_2022)
  MARKET_2022 = 0

In [ ]:
MARKET_2023_LIST = []
directory_상권_2023: List = [item.split(".csv")[0] + "_2023.csv" for item in directory_상권_2022[1:]]

for index, data in zip(list(range(len(directory_상권_2023))), directory_상권_2023):
  MARKET_2023 = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/상권_2023/",
    file_name_extension = data,
    unicode = "euc-kr", columnTF = False)
  MARKET_2023_LIST.append(MARKET_2023)
  MARKET_2023 = 0

In [109]:
display(directory_상권_2022)
MARKET_2022_LIST[0]

['서울시 상권분석서비스(상권영역).csv',
 '서울시 상권분석서비스(상권배후지-아파트).csv',
 '서울시 상권분석서비스(상권-소득소비).csv',
 '서울시 상권분석서비스(상권-아파트).csv',
 '서울시 상권분석서비스(상권배후지-직장인구).csv',
 '서울시 상권분석서비스(상권배후지-점포).csv',
 '서울시 상권분석서비스(상권-점포).csv',
 '서울시 상권분석서비스(상권배후지-집객시설).csv',
 '서울시 상권분석서비스(버스 총 승차 승객수 정보).csv',
 '서울시 상권분석서비스(상권배후지-생활인구).csv',
 '서울시 상권분석서비스(지하철 총 승차 승객수 정보).csv',
 '서울시 상권분석서비스(상권-생활인구).csv',
 '서울시 상권분석서비스(상권-직장인구).csv',
 '서울시 상권분석서비스(행정동별 상권변화지표).csv']

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보
0,202212,A,골목상권,3110008,배화여자대학교(박노수미술관),197093,453418,11110,11110515,NaN
1,202212,A,골목상권,3110009,자하문터널,196991,455057,11110,11110550,NaN
2,202212,A,골목상권,3110010,평창동서측,197064,456643,11110,11110560,NaN
3,202212,A,골목상권,3110017,정독도서관,198581,453781,11110,11110600,NaN
4,202212,A,골목상권,3110018,중앙고등학교,198883,453690,11110,11110600,NaN
...,...,...,...,...,...,...,...,...,...,...
1645,202212,U,관광특구,3001492,명동 남대문 북창동 다동 무교동 관광특구,198397,451614,11140,11140520,NaN
1646,202212,U,관광특구,3001493,동대문패션타운 관광특구,200996,451976,11140,11140590,NaN
1647,202212,U,관광특구,3001494,종로?청계 관광특구,199796,452274,11110,11110615,NaN
1648,202212,U,관광특구,3001495,잠실 관광특구,210188,446334,11710,11710566,NaN


In [110]:
display(directory_상권_2023)
MARKET_2023_LIST[6]

['서울시 상권분석서비스(상권배후지-아파트)_2023.csv',
 '서울시 상권분석서비스(상권-소득소비)_2023.csv',
 '서울시 상권분석서비스(상권-아파트)_2023.csv',
 '서울시 상권분석서비스(상권배후지-직장인구)_2023.csv',
 '서울시 상권분석서비스(상권배후지-점포)_2023.csv',
 '서울시 상권분석서비스(상권-점포)_2023.csv',
 '서울시 상권분석서비스(상권배후지-집객시설)_2023.csv',
 '서울시 상권분석서비스(버스 총 승차 승객수 정보)_2023.csv',
 '서울시 상권분석서비스(상권배후지-생활인구)_2023.csv',
 '서울시 상권분석서비스(지하철 총 승차 승객수 정보)_2023.csv',
 '서울시 상권분석서비스(상권-생활인구)_2023.csv',
 '서울시 상권분석서비스(상권-직장인구)_2023.csv',
 '서울시 상권분석서비스(행정동별 상권변화지표)_2023.csv']

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,집객시설_수,관공서_수,은행_수,종합병원_수,...,대학교_수,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,철도_역_수,버스_터미널_수,지하철_역_수,버스_정거장_수
0,2023,1,A,골목상권,3110363,송천동주민센터,85,3.0000,2.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15
1,2023,1,A,골목상권,3110754,삼성산자연공원(서울금천초등학교),123,4.0000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46
2,2023,1,A,골목상권,3110906,서문여고,42,1.0000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10
3,2023,1,A,골목상권,3110846,남성역 1번,130,6.0000,2.0000,NaN,...,NaN,NaN,1.0000,NaN,NaN,NaN,NaN,NaN,1.0000,30
4,2023,1,A,골목상권,3110778,구로디지털단지역 4번,166,4.0000,10.0000,1.0000,...,1.0000,NaN,NaN,NaN,4.0000,NaN,NaN,NaN,1.0000,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085,2023,1,A,골목상권,3110282,국민대학교앞,62,3.0000,1.0000,NaN,...,26.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15
1086,2023,1,A,골목상권,3110312,안암역 2번,133,1.0000,5.0000,1.0000,...,72.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0000,23
1087,2023,1,A,골목상권,3110825,신대방삼거리역 1번,192,2.0000,5.0000,NaN,...,NaN,NaN,1.0000,NaN,NaN,NaN,NaN,NaN,1.0000,58
1088,2023,1,A,골목상권,3110864,신림우방아파트,49,2.0000,1.0000,NaN,...,NaN,NaN,1.0000,NaN,1.0000,NaN,NaN,NaN,1.0000,16
